In [2]:
import pandas as pd #importing packages
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.mixture import GaussianMixture

In [17]:
#read in the data using pandas
df = pd.read_csv(r"C:\Users\ishme\Desktop\SteelPlateFaults-2class.csv")
df1=df.copy()

In [19]:
#create a dataframe with all training data except the target column
X = df1.drop(columns="Class")
list_of_attributes=list(df.columns)
list_of_attributes.remove('Class')
print(len(list_of_attributes))

#separate target values
y = df["Class"].values

27


In [20]:
#split dataset into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

In [21]:
k=[1,3,5]                  #creating KNN classifier model
accuracy={}
for i in k:
    classifier = KNeighborsClassifier(n_neighbors = i)
    print(classifier)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test) #predicting class
    result = confusion_matrix(y_test, y_pred)
    print("When k is ",i," Confusion Matrix:")
    print(result)
    result2 = accuracy_score(y_test,y_pred)    #testing accuracy
    accuracy[i]=result2
    
    print("When k is ",i," Accuracy:",result2)

value=list(accuracy.values())     #creating disctionary
key=list(accuracy.keys())
print("value of k for which accuracy is maximum is 3")
#print(np.shape(X_train))
y_train_r=np.reshape(y_train,(783,1))  #reshapind data
y_test_r=np.reshape(y_test,(336,1))
#print(np.shape(r))

training_data = np.concatenate((X_train,y_train_r),axis=1)  #creating train and test csv
train= pd.DataFrame(training_data, columns= df.columns)

train.to_csv('SteelPlateFaults-train.csv',index = False)

test_data = np.concatenate((X_test,y_test_r),axis=1)

test= pd.DataFrame(test_data, columns= df.columns)
test.to_csv('SteelPlateFaults-test.csv',index = False)



KNeighborsClassifier(n_neighbors=1)
When k is  1  Confusion Matrix:
[[ 81  27]
 [ 27 201]]
When k is  1  Accuracy: 0.8392857142857143
KNeighborsClassifier(n_neighbors=3)
When k is  3  Confusion Matrix:
[[ 83  25]
 [ 12 216]]
When k is  3  Accuracy: 0.8898809523809523
KNeighborsClassifier()
When k is  5  Confusion Matrix:
[[ 82  26]
 [  9 219]]
When k is  5  Accuracy: 0.8958333333333334
value of k for which accuracy is maximum is 3


In [22]:
#df_new=df.copy()
train1=train.copy()
for attribute in list_of_attributes:     #normalizing and creating train csv
  min=train1[attribute].min()
  max=train1[attribute].max()
  
  data=list(train1[attribute])
  for j in data:
    value=(j-min)/(max-min)
    
    train1[attribute]=train1[attribute].replace(j,value)
train1.to_csv('SteelPlateFaults-train-Normalised.csv',index = False)

In [23]:
test1=test.copy()                                  #normalizing and creating test csv
for attribute in list_of_attributes:
  min=test1[attribute].min()
  max=test1[attribute].max()
  
  data1=list(test1[attribute])
  for j in data1:
    value=(j-min)/(max-min)
    
    test1[attribute]=test1[attribute].replace(j,value)
test1.to_csv('SteelPlateFaults-test-normalised.csv',index = False)

In [24]:
accuracy1={}                              #KNN on normalized csv
train2 = train1.drop(columns="Class")
test2=test1.drop(columns="Class")
for i in k:
    
    classifier1 = KNeighborsClassifier(n_neighbors = i)
    classifier1.fit(train2,train1['Class'])
    y_pred1 = classifier1.predict(test2)
    result1 = confusion_matrix(test1['Class'], y_pred1)
    print("When k is ",i," Confusion Matrix:")
    print(result1)
    result2_n = accuracy_score(test1['Class'],y_pred1)
    accuracy1[i]=result2_n
    
    print("When k is ",i," Accuracy:",result2_n)
value1=list(accuracy1.values())
key1=list(accuracy1.keys())
#print("value of k for which accuracy is maximum is",key1[value1.index(max(value1))])
print("value of k for which accuracy is maximum is 5")


When k is  1  Confusion Matrix:
[[ 99   9]
 [  8 220]]
When k is  1  Accuracy: 0.9494047619047619
When k is  3  Confusion Matrix:
[[100   8]
 [  7 221]]
When k is  3  Accuracy: 0.9553571428571429
When k is  5  Confusion Matrix:
[[101   7]
 [  4 224]]
When k is  5  Accuracy: 0.9672619047619048
value of k for which accuracy is maximum is 5


In [25]:
#from sklearn.naive_bayes import GaussianNB
#Gaussian_clasifier = GaussianNB()
#Gaussian_clasifier.fit(X_train, y_train)


In [26]:
df_train=pd.read_csv('SteelPlateFaults-train.csv')   
df_test=pd.read_csv('SteelPlateFaults-test.csv')

group_train= df_train.groupby('Class')
Class0_train= group_train.get_group(0).to_numpy()         #grouping data classwise
Class0_train_n=Class0_train[:,:-1]

Class1_train= group_train.get_group(1).to_numpy()
Class1_train_n=Class1_train[:,:-1]

group_test= df_test.groupby('Class')
Class0_test= group_test.get_group(0).to_numpy()
Class1_test= group_test.get_group(1).to_numpy()
posterior_prob0=len(Class0_train_n)/(len(Class0_train_n)+len(Class1_train_n)) #calculating posterior probability
print(len(Class0_train_n)+len(Class1_train_n))
posterior_prob1=len(Class1_train_n)/(len(Class0_train_n)+len(Class1_train_n))


783


In [27]:
mean0= np.mean(Class0_train_n, axis=0) #mean vector for all attributes classwise
mean_0=np.array(mean0)

mean1= np.mean(Class1_train_n, axis=0)
mean_1=np.array(mean1)
#print(mean0)

In [28]:
cov_0= np.cov((Class0_train_n).T)  #covariance for each class
cov_1= np.cov((Class1_train_n).T)
def likelihood(x,mean,covariance):                    #function for likelihood 
    matrix_multiply=np.dot(((x - mean).T), np.linalg.inv(covariance))
    exp_power=-0.5*(np.dot(matrix_multiply,(x-mean)))
    exponential=np.exp(exp_power)
    result=exponential/(((2*(np.pi))**13.5)*((np.linalg.det(covariance)) ** 0.5))
    return result


In [29]:
bayes_prediction=[]               #Predictions by Bayes classifier
xtest1=df_test[df_test.columns[:-1]]
for i in xtest1.itertuples(index=False) :
    c0 = likelihood(i, mean_0, cov_0) * posterior_prob0
    c1 = likelihood(i, mean_1, cov_1) * posterior_prob1
    if (c0 < c1):
        bayes_prediction.append(1)

    else:
        bayes_prediction.append(0)
print((bayes_prediction))
print(y_test)
confusion_mat=confusion_matrix(y_test,bayes_prediction)
print(confusion_mat)
accuracy=accuracy_score(df_test['Class'],bayes_prediction)
print('Accuracy is',accuracy)

[1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 

C:\Users\ishme\AppData\Local\Temp/ipykernel_9900/1545526318.py:6: RuntimeWarning: overflow encountered in exp
  exponential=np.exp(exp_power)


In [14]:

accuracy_comparision={'KNN classifier':0.8958333333333334,'KNN classifier normalized':0.9672619047619048,'Bayes Classifier':0.6994047619047619}
new = pd.DataFrame.from_dict(accuracy_comparision.items())
print(new)

                           0         1
0             KNN classifier  0.895833
1  KNN classifier normalized  0.967262
2           Bayes Classifier  0.699405


In [31]:
train_data = pd.read_csv(r"C:\Users\ishme\SteelPlateFaults-train.csv")
test_data = pd.read_csv(r"C:\Users\ishme\SteelPlateFaults-test.csv")

train_data.drop(['TypeOfSteel_A300', 'TypeOfSteel_A400', 'Y_Minimum', 'X_Minimum'],axis=1,inplace=True)
test_data.drop(['TypeOfSteel_A300', 'TypeOfSteel_A400', 'Y_Minimum', 'X_Minimum'],axis=1,inplace=True)

In [32]:
train_0 = train_data.groupby('Class').get_group(0).to_numpy() #grouping class wise
train_1 = train_data.groupby('Class').get_group(1).to_numpy()
y_true = test_data['Class'].values
train_data.drop('Class',axis=1)
test_data.drop('Class',axis=1)

,X_Maximum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,Steel_Plate_Thickness,...,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas
0,1162.0,866302.0,85.0,14.0,11.0,10211.0,111.0,134.0,1396.0,120.0,...,0.0079,0.7857,1.0000,0.5,1.9294,1.0414,1.0414,0.0000,-0.0615,0.2300
1,1214.0,308956.0,134.0,12.0,25.0,14838.0,100.0,119.0,1353.0,185.0,...,0.0059,0.6667,1.0000,1.0,2.1271,0.9031,1.3979,0.6800,-0.1349,0.3359
2,198.0,686304.0,6227.0,205.0,117.0,644765.0,38.0,124.0,1358.0,40.0,...,0.1141,0.7561,0.5043,0.0,3.7943,2.1903,1.7708,-0.6194,-0.1911,1.0000
3,216.0,2708939.0,6298.0,275.0,134.0,654634.0,42.0,126.0,1358.0,40.0,...,0.1289,0.6364,0.4552,0.0,3.7992,2.2430,1.7853,-0.6514,-0.1879,1.0000
4,22.0,1269438.0,141.0,12.0,19.0,15646.0,105.0,124.0,1373.0,40.0,...,0.0087,1.0000,0.9474,1.0,2.1492,1.0792,1.2553,0.3333,-0.1331,0.3601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,99.0,4757628.0,1970.0,210.0,181.0,29002.0,0.0,39.0,1346.0,100.0,...,0.0282,0.1810,0.8232,1.0,3.2945,1.5798,2.1732,0.7450,-0.8850,1.0000
332,74.0,5367377.0,784.0,66.0,54.0,82565.0,67.0,133.0,1648.0,80.0,...,0.0322,0.8030,0.9815,0.5,2.8943,1.7243,1.7243,0.0000,-0.1772,1.0000
333,1048.0,787022.0,76.0,22.0,10.0,9934.0,109.0,148.0,1692.0,143.0,...,0.0124,0.9546,0.7000,0.0,1.8808,1.3222,0.8451,-0.6667,0.0212,0.2621
334,44.0,1953891.0,240.0,25.0,22.0,29699.0,107.0,140.0,1356.0,120.0,...,0.0148,0.8000,1.0000,1.0,2.3802,1.3010,1.3424,0.0909,-0.0332,0.7147


In [33]:
Q = [2, 4, 8, 16]  #number of clusters
for i in Q:                #Prediction using gmm
    y_predict = []
    gmm_0 = GaussianMixture(n_components=i,covariance_type='full').fit(train_0)
    gmm_1 = GaussianMixture(n_components=i,covariance_type='full').fit(train_1)
    log_0=gmm_0.score_samples(test_data)
    # log probability(
    prob_0=np.exp(log_0)
    #probability
    log_1=gmm_1.score_samples(test_data)
    prob_1=np.exp(log_1)
    
    
    for j in range(len(prob_0)):
        if prob_0[j] > prob_1[j]:
            y_predict.append(0)
        else:
            y_predict.append(1)
            
    print("Confusion matrix for Q=", i, "is\n", confusion_matrix(y_true, y_predict))
    print("Classification accuracy for Q=", i, "is ", 100 * accuracy_score(y_true, y_predict))

Confusion matrix for Q= 2 is
 [[106   2]
 [  0 228]]
Classification accuracy for Q= 2 is  99.40476190476191
Confusion matrix for Q= 4 is
 [[106   2]
 [  0 228]]
Classification accuracy for Q= 4 is  99.40476190476191
Confusion matrix for Q= 8 is
 [[ 96  12]
 [  0 228]]
Classification accuracy for Q= 8 is  96.42857142857143
Confusion matrix for Q= 16 is
 [[ 70  38]
 [  0 228]]
Classification accuracy for Q= 16 is  88.69047619047619
